## Fairness processing

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from sklego.linear_model import EqualOpportunityClassifier
from fairlearn.postprocessing import ThresholdOptimizer

from utils import read_diabetes_dataset
from utils import statistical_parity
from utils import average_odds
from utils import average_predictive_value
from utils import theil_index

from utils import Weighted
from utils import CounterfactualPreProcessing

import pandas as pd
import numpy as np

### Dataset

In [2]:
X, y = read_diabetes_dataset(binary=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)


In [3]:
# Male: 1, Female: 0
sens_variable = 'gender'

Z_train = X_train[sens_variable]
Z_test = X_test[sens_variable]

# sens_columns = ['race_AfricanAmerican', 'race_Asian', 'race_Caucasian', 'race_Hispanic','race_Other']
# Z_train = np.argmax(X_train[sens_columns].values, axis=1)
# Z_test = np.argmax(X_test[sens_columns].values, axis=1)

### Without Mitigation

In [4]:
# Logistic Regression
lr = LogisticRegression(class_weight=None, max_iter=10**3)
lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

print('Accuracy', accuracy_score(y_train, y_train_pred))
print('ROC', roc_auc_score(y_train, y_train_pred))
print('Statistical parity', statistical_parity(y_train, y_train_pred, Z_train))
print('Average odds', average_odds(y_train, y_train_pred, Z_train))
print('Average predictive value', average_predictive_value(y_train, y_train_pred, Z_train))
print('Theil index', theil_index(y_train, y_train_pred))

print("----------------------")

print('Accuracy', accuracy_score(y_test, y_test_pred))
print('ROC', roc_auc_score(y_test, y_test_pred))
print('Statistical parity', statistical_parity(y_test, y_test_pred, Z_test))
print('Average odds', average_odds(y_test, y_test_pred, Z_test))
print('Average predictive value', average_predictive_value(y_test, y_test_pred, Z_test))
print('Theil index', theil_index(y_test, y_test_pred))


Accuracy 0.6260437058048021
ROC 0.5530684943265511
[0] Statistical parity [-0.02823257]
[0] Average odds [-0.02776224]
[0] Average predictive value [-0.00958463]
Theil index 0.41195578101024366
----------------------
Accuracy 0.6222501864280388
ROC 0.5515052717936609
[0] Statistical parity [-0.03763251]
[0] Average odds [-0.03691255]
[0] Average predictive value [0.01194706]
Theil index 0.41560613437135546


In [5]:
# # Decision Tree
# dt = DecisionTreeClassifier(max_depth=7, random_state=123, class_weight=None)
# dt.fit(X_train, y_train)

# y_train_pred = dt.predict(X_train)
# y_test_pred = dt.predict(X_test)

# print('Accuracy', accuracy_score(y_train, y_train_pred))
# print('ROC', roc_auc_score(y_train, y_train_pred))
# print('Statistical parity', statistical_parity(y_train, y_train_pred, Z_train))
# print('Average odds', average_odds(y_train, y_train_pred, Z_train))
# print('Average predictive value', average_predictive_value(y_train, y_train_pred, Z_train))
# print('Theil index', theil_index(y_train, y_train_pred))

# print("----------------------")

# print('Accuracy', accuracy_score(y_test, y_test_pred))
# print('ROC', roc_auc_score(y_test, y_test_pred))
# print('Statistical parity', statistical_parity(y_test, y_test_pred, Z_test))
# print('Average odds', average_odds(y_test, y_test_pred, Z_test))
# print('Average predictive value', average_predictive_value(y_test, y_test_pred, Z_test))
# print('Theil index', theil_index(y_test, y_test_pred))


### Pre-processing

In [6]:
sens_position = np.argwhere(X.columns==sens_variable)[0][0]
sens_feature_ids = [sens_position]

- Weighted

In [7]:
lr_weighted = Weighted(LogisticRegression(class_weight=None, max_iter=10**3), sensitive_feature_ids=sens_feature_ids)
lr_weighted.fit(X_train.values, y_train)

y_train_pred = lr_weighted.predict(X_train.values)
y_test_pred = lr_weighted.predict(X_test.values)

print('Accuracy', accuracy_score(y_train, y_train_pred))
print('ROC', roc_auc_score(y_train, y_train_pred))
print('Statistical parity', statistical_parity(y_train, y_train_pred, Z_train))
print('Average odds', average_odds(y_train, y_train_pred, Z_train))
print('Average predictive value', average_predictive_value(y_train, y_train_pred, Z_train))
print('Theil index', theil_index(y_train, y_train_pred))

print("----------------------")

print('Accuracy', accuracy_score(y_test, y_test_pred))
print('ROC', roc_auc_score(y_test, y_test_pred))
print('Statistical parity', statistical_parity(y_test, y_test_pred, Z_test))
print('Average odds', average_odds(y_test, y_test_pred, Z_test))
print('Average predictive value', average_predictive_value(y_test, y_test_pred, Z_test))
print('Theil index', theil_index(y_test, y_test_pred))


/home/maldonadoq/University/Unicamp/Teams/MO810/diabetes_fairness/utils.py:279: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  X[:, self.sensitive_feature_ids], y[:, np.newaxis], axis=1)


Accuracy 0.6253046222683872
ROC 0.5521743793818592
[0] Statistical parity [0.00117495]
[0] Average odds [0.00299513]
[0] Average predictive value [-0.02161604]
Theil index 0.4129778877567931
----------------------
Accuracy 0.6216442953020134
ROC 0.5501138897666888
[0] Statistical parity [-0.00843652]
[0] Average odds [-0.00673999]
[0] Average predictive value [-0.00157209]
Theil index 0.41824692244026274


- Counterfactual

In [8]:
lr_counter = CounterfactualPreProcessing(LogisticRegression(class_weight=None, max_iter=10**3), sensitive_feature_ids=sens_feature_ids)
lr_counter.fit(X_train.values, y_train)

y_train_pred = lr_counter.predict(X_train.values)
y_test_pred = lr_counter.predict(X_test.values)

print('Accuracy', accuracy_score(y_train, y_train_pred))
print('ROC', roc_auc_score(y_train, y_train_pred))
print('Statistical parity', statistical_parity(y_train, y_train_pred, Z_train))
print('Average odds', average_odds(y_train, y_train_pred, Z_train))
print('Average predictive value', average_predictive_value(y_train, y_train_pred, Z_train))
print('Theil index', theil_index(y_train, y_train_pred))

print("----------------------")

print('Accuracy', accuracy_score(y_test, y_test_pred))
print('ROC', roc_auc_score(y_test, y_test_pred))
print('Statistical parity', statistical_parity(y_test, y_test_pred, Z_test))
print('Average odds', average_odds(y_test, y_test_pred, Z_test))
print('Average predictive value', average_predictive_value(y_test, y_test_pred, Z_test))
print('Theil index', theil_index(y_test, y_test_pred))


(100124, 32) (100124,)
Accuracy 0.6250249690383924
ROC 0.5518911158815594
[0] Statistical parity [-0.00435518]
[0] Average odds [-0.002853]
[0] Average predictive value [-0.01978334]
Theil index 0.4132204021170181
----------------------
Accuracy 0.6215044742729307
ROC 0.5502475624798616
[0] Statistical parity [-0.01472042]
[0] Average odds [-0.01322206]
[0] Average predictive value [0.00130283]
Theil index 0.41762123723167116


### In-processing

- Fairness mitigation

In [9]:
Xn_train = X_train.drop([sens_variable], axis=1)
Xn_test = X_test.drop([sens_variable], axis=1)

In [10]:
# Logistic Regression
lr = LogisticRegression(class_weight=None, max_iter=10**3)
lr.fit(Xn_train, y_train)

y_train_pred = lr.predict(Xn_train)
y_test_pred = lr.predict(Xn_test)

print('Accuracy', accuracy_score(y_train, y_train_pred))
print('ROC', roc_auc_score(y_train, y_train_pred))
print('Statistical parity', statistical_parity(y_train, y_train_pred, Z_train))
print('Average odds', average_odds(y_train, y_train_pred, Z_train))
print('Average predictive value', average_predictive_value(y_train, y_train_pred, Z_train))
print('Theil index', theil_index(y_train, y_train_pred))

print("----------------------")

print('Accuracy', accuracy_score(y_test, y_test_pred))
print('ROC', roc_auc_score(y_test, y_test_pred))
print('Statistical parity', statistical_parity(y_test, y_test_pred, Z_test))
print('Average odds', average_odds(y_test, y_test_pred, Z_test))
print('Average predictive value', average_predictive_value(y_test, y_test_pred, Z_test))
print('Theil index', theil_index(y_test, y_test_pred))


Accuracy 0.6249850185769645
ROC 0.5518579241662116
[0] Statistical parity [-0.00475122]
[0] Average odds [-0.00328614]
[0] Average predictive value [-0.01976025]
Theil index 0.41323594808904546
----------------------
Accuracy 0.6215510812826249
ROC 0.5502864518008541
[0] Statistical parity [-0.01463283]
[0] Average odds [-0.01314833]
[0] Average predictive value [0.00113808]
Theil index 0.41760316288110444


In [11]:
# # Decision Tree
# dt = DecisionTreeClassifier(max_depth=7, random_state=123, class_weight=None)
# dt.fit(Xn_train, y_train)

# y_train_pred = dt.predict(Xn_train)
# y_test_pred = dt.predict(Xn_test)

# print('Accuracy', accuracy_score(y_train, y_train_pred))
# print('ROC', roc_auc_score(y_train, y_train_pred))
# print('Statistical parity', statistical_parity(y_train, y_train_pred, Z_train))
# print('Average odds', average_odds(y_train, y_train_pred, Z_train))
# print('Average predictive value', average_predictive_value(y_train, y_train_pred, Z_train))
# print('Theil index', theil_index(y_train, y_train_pred))


- Counterfactual appends

In [12]:
X_train_cf =  X_train.copy()
X_train_cf[sens_variable] = X_train_cf[sens_variable].replace({0:1, 1:0})
X_train_cf = pd.concat([X_train_cf, X_train])
y_train_cf = pd.concat([y_train, y_train])

X_test_cf =  X_test.copy()
X_test_cf[sens_variable] = X_test_cf[sens_variable].replace({0:1, 1:0})
X_test_cf = pd.concat([X_test_cf, X_test])
y_test_cf = pd.concat([y_test, y_test])

In [13]:
# Logistic Regression
lr = LogisticRegression(class_weight=None, max_iter=10**3)
lr.fit(X_train_cf, y_train_cf)

y_train_pred = lr.predict(X_train_cf)
y_test_pred = lr.predict(X_test_cf)

print('Accuracy', accuracy_score(y_train_cf, y_train_pred))
print('ROC', roc_auc_score(y_train_cf, y_train_pred))
print('Statistical parity', statistical_parity(y_train_cf, y_train_pred, Z_train))
print('Average odds', average_odds(y_train_cf, y_train_pred, Z_train))
print('Average predictive value', average_predictive_value(y_train_cf, y_train_pred, Z_train))
print('Theil index', theil_index(y_train_cf, y_train_pred))

print("----------------------")

print('Accuracy', accuracy_score(y_test_cf, y_test_pred))
print('ROC', roc_auc_score(y_test_cf, y_test_pred))
print('Statistical parity', statistical_parity(y_test_cf, y_test_pred, Z_test))
print('Average odds', average_odds(y_test_cf, y_test_pred, Z_test))
print('Average predictive value', average_predictive_value(y_test_cf, y_test_pred, Z_test))
print('Theil index', theil_index(y_test_cf, y_test_pred))

Accuracy 0.6249750309616076
ROC 0.5518920608147619
[0] Statistical parity [-0.00488636]
[0] Average odds [-0.00339224]
[0] Average predictive value [-0.01944092]
Theil index 0.41312842615755935
----------------------
Accuracy 0.6216209917971663
ROC 0.5504121851951526
[0] Statistical parity [-0.01523386]
[0] Average odds [-0.01374838]
[0] Average predictive value [0.00151627]
Theil index 0.41739288432417876


In [14]:
# # Decision Tree
# dt = DecisionTreeClassifier(max_depth=7, random_state=123, class_weight=None)
# dt.fit(X_train_cf, y_train_cf)

# y_train_pred = dt.predict(X_train_cf)
# y_test_pred = dt.predict(X_test_cf)

# print('Accuracy', accuracy_score(y_train_cf, y_train_pred))
# print('ROC', roc_auc_score(y_train_cf, y_train_pred))
# print('Statistical parity', statistical_parity(y_train_cf, y_train_pred, Z_train))
# print('Average odds', average_odds(y_train_cf, y_train_pred, Z_train))
# print('Average predictive value', average_predictive_value(y_train_cf, y_train_pred, Z_train))
# print('Theil index', theil_index(y_train_cf, y_train_pred))


### Post-processing

In [15]:
Z_train = X_train[sens_variable]==0
Z_test = X_test[sens_variable]==0

- Model

In [16]:
lr = LogisticRegression(class_weight=None, max_iter=10**3)
lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

print('Accuracy', accuracy_score(y_train, y_train_pred))
print('ROC', roc_auc_score(y_train, y_train_pred))
print('Statistical parity', statistical_parity(y_train, y_train_pred, Z_train))
print('Average odds', average_odds(y_train, y_train_pred, Z_train))
print('Average predictive value', average_predictive_value(y_train, y_train_pred, Z_train))
print('Theil index', theil_index(y_train, y_train_pred))

print("----------------------")

print('Accuracy', accuracy_score(y_test, y_test_pred))
print('ROC', roc_auc_score(y_test, y_test_pred))
print('Statistical parity', statistical_parity(y_test, y_test_pred, Z_test))
print('Average odds', average_odds(y_test, y_test_pred, Z_test))
print('Average predictive value', average_predictive_value(y_test, y_test_pred, Z_test))
print('Theil index', theil_index(y_test, y_test_pred))

Accuracy 0.6260437058048021
ROC 0.5530684943265511
[True] Statistical parity [-0.02823257]
[True] Average odds [-0.02776224]
[True] Average predictive value [-0.00958463]
Theil index 0.41195578101024366
----------------------
Accuracy 0.6222501864280388
ROC 0.5515052717936609
[True] Statistical parity [-0.03763251]
[True] Average odds [-0.03691255]
[True] Average predictive value [0.01194706]
Theil index 0.41560613437135546


- Optimizer

In [17]:
postprocess_est = ThresholdOptimizer(
    estimator=lr,
    constraints="false_negative_rate_parity",
    objective="balanced_accuracy_score",
    prefit=True,
    predict_method='predict_proba')
postprocess_est.fit(X_train, y_train, sensitive_features=Z_train)

y_train_pred = postprocess_est.predict(X_train, sensitive_features=Z_train)
y_test_pred = postprocess_est.predict(X_test, sensitive_features=Z_test)


print('Accuracy', accuracy_score(y_train, y_train_pred))
print('ROC', roc_auc_score(y_train, y_train_pred))
print('Statistical parity', statistical_parity(y_train, y_train_pred, Z_train))
print('Average odds', average_odds(y_train, y_train_pred, Z_train))
print('Average predictive value', average_predictive_value(y_train, y_train_pred, Z_train))
print('Theil index', theil_index(y_train, y_train_pred))

print("----------------------")

print('Accuracy', accuracy_score(y_test, y_test_pred))
print('ROC', roc_auc_score(y_test, y_test_pred))
print('Statistical parity', statistical_parity(y_test, y_test_pred, Z_test))
print('Average odds', average_odds(y_test, y_test_pred, Z_test))
print('Average predictive value', average_predictive_value(y_test, y_test_pred, Z_test))
print('Theil index', theil_index(y_test, y_test_pred))

Accuracy 0.5784427310135432
ROC 0.58531138351355
[True] Statistical parity [0.00275412]
[True] Average odds [0.0048536]
[True] Average predictive value [-0.01879692]
Theil index 0.22287843572520022
----------------------
Accuracy 0.5783463832960477
ROC 0.5846588091394347
[True] Statistical parity [0.01073702]
[True] Average odds [0.01458106]
[True] Average predictive value [-0.01342983]
Theil index 0.22520008264753213
